In [1]:
import json
from preprocess.vn_validator import validate_and_fix_vietnamese_text
from preprocess.text_normalizer import (
    strip_html,
    normalize_spacing,
    remove_code_artifacts
)

data_path = "./data/raw/news_dataset.json"

with open(data_path, "r") as data_file:
    data = json.load(data_file)
print(f"Loaded {len(data)} records from {data_path}")

Loaded 184539 records from ./data/raw/news_dataset.json


In [2]:
# Inspect a sample
data[0]

{'id': 218270,
 'author': '',
 'content': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường

In [3]:
def preprocess_content(content: str) -> str:
    # Normalize content into Unicode NFC
    content = validate_and_fix_vietnamese_text(content)[0]
    
    # Remove HTML tags
    content = strip_html(content)
    
    # Normalize spacing
    content = normalize_spacing(content)
    
    # Remove common code artifacts
    content = remove_code_artifacts(content)
    
    return content

def preprocess_item(item: dict) -> dict:
    # First, we preprocess the main content of each piece of news.
    preprocessed_item: dict = {}
    preprocessed_item["content"] = preprocess_content(item["content"])
    
    # Topic is treated as the label for later classification task.
    preprocessed_item["topic"] = item["topic"]
    
    # Then, we store the metadata of each piece of news.
    metadata_fields = ["source", "topic", "crawled_at"]
    preprocessed_item["metadata"]: dict = {field: item[field] for field in metadata_fields}
    
    return preprocessed_item

In [4]:
preprocess_item(data[0])

{'content': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng, cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, 